In [3]:
import pandas as pd
df = pd.read_csv(r'PhiUSIIL_Phishing_URL_Dataset.csv',encoding="utf-8")

In [4]:
df = df.drop(['Bank', 'Pay', 'Crypto','URLSimilarityIndex'], axis=1)

In [5]:
df.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,HasHiddenFields,HasPasswordField,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,1.000000,0.522907,0.061933,...,1,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,0.666667,0.032650,0.050207,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,0.866667,0.028555,0.064129,...,1,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,1.000000,0.522907,0.057606,...,1,0,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,1.000000,0.079963,0.059441,...,1,0,1,244,15,34,72,1,85,1


In [6]:
def vishnu(x):
    return len(x)
df['URLLength'] = df['URL'].apply(vishnu)

In [7]:
import socket
from urllib.parse import urlparse

def is_ip(domain):
    try:
        # Validate IPv4 address
        socket.inet_aton(domain)
        return True
    except socket.error:
        try:
            # Validate IPv6 address
            socket.inet_pton(socket.AF_INET6, domain)
            return 1
        except (socket.error, ValueError):
            return 0
df['IsDomainIP'] = df.URL.apply(is_ip)

In [8]:
import requests
import os
from urllib.parse import urlparse

def url_similarity_index(url):
    """Computes the similarity index of a given URL against the top websites."""

    def get_min(src, tar):
        return (src, tar, len(src)) if len(src) <= len(tar) else (tar, src, len(tar))

    def get_url_similarity_index(src, tar):
        X, Y, n = get_min(src, tar)
        N = max(len(src), len(tar))
        SI = 0
        a = 50 / N
        nsum = (N * (N + 1)) // 2

        i = 0
        while i < n:
            if X[i] == Y[i]:
                SI += a + (50 * (N - i)) / nsum
            else:
                Y = Y[:i] + Y[i + 1:]
                X, Y, n = get_min(src, tar)
                i -= 1  # Adjust index after removal
            i += 1

        return SI

    def extract_domain(url):
        parsed_url = urlparse(url)
        return parsed_url.netloc or parsed_url.path

    def get_top_websites():
        OPR_API_KEY = os.getenv("OPR_API_KEY")
        if not OPR_API_KEY:
            raise ValueError("Error: OPR_API_KEY is not set. Please set the API key as an environment variable.")

        TOP_SITES_URL = "https://openpagerank.com/api/v1.0/getPageRank"
        headers = {"API-OPR": OPR_API_KEY}

        response = requests.get(TOP_SITES_URL, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if "response" in data and isinstance(data["response"], list):
                return [entry["domain"] for entry in data["response"] if "domain" in entry]
        elif response.status_code == 403:
            raise ValueError("Error: Invalid API key. Please check your OPR_API_KEY.")
        else:
            raise ValueError(f"Error fetching top websites: {response.status_code} - {response.text}")

    top_websites = get_top_websites()
    src_domain = extract_domain(url)
    most_similar_url = None
    highest_similarity = -1

    for website in top_websites:
        similarity = get_url_similarity_index(src_domain, website)
        if similarity > highest_similarity:
            highest_similarity = similarity
            most_similar_url = website

    if most_similar_url:
        print(f"Most similar URL: {most_similar_url}")
        print(f"Similarity Index: {highest_similarity}")
    else:
        print("No similar URLs found.")

    return highest_similarity

In [9]:
def char_continuation_rate(url):
    if len(url) <= 1:
        return 0.0
    count = sum(1 for i in range(len(url) - 1) if url[i] == url[i + 1])
    return round(count / len(url), 3)
df['CharContinuationRate'] = df['URL'].apply(char_continuation_rate)

In [10]:
from collections import Counter

def url_char_prob(url):

    # Handle edge case: empty URL
    if not url:
        return {}

    # Count occurrences of each character
    char_counts = Counter(url)

    # Calculate total length of the URL
    total_length = len(url)

    # Calculate probabilities
    char_probabilities = {char: count / total_length for char, count in char_counts.items()}

    return char_probabilities

In [11]:
def max_char_prob(url):
    char_probs = url_char_prob(url)
    return max(char_probs.values()) if char_probs else 0.0
df['URLCharProb'] = df["URL"].apply(max_char_prob)

In [12]:
def tld_vishnu(tld):
    return len(tld)
df['TLDLength'] = df['TLD'].apply(tld_vishnu)

In [13]:
!pip install tldextract


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import tldextract
import pandas as pd

# Define the function to count subdomains
def count_subdomains(url):
    extracted = tldextract.extract(url)
    subdomain = extracted.subdomain
    return subdomain.count(".") + 1 if subdomain else 0
df['NoOfSubDomain'] = df['URL'].apply(count_subdomains)

In [15]:
import re
import pandas as pd
def has_obfuscation(url):
    return 1 if re.search(r"%[0-9A-Fa-f]{2}|@", url) else 0
df['HasObfuscation'] = df['URL'].apply(has_obfuscation)

In [16]:
import pandas as pd
def count_obfuscated_chars(url):
    return sum(url.count(c) for c in ["%", "@"])
df['NoOfObfuscatedChar'] = df['URL'].apply(count_obfuscated_chars)

In [17]:
import pandas as pd
def count_obfuscated_chars(url):
    return sum(url.count(c) for c in ["%", "@"])
def obfuscation_ratio(url):
    obf_chars = count_obfuscated_chars(url)
    return obf_chars / len(url) if len(url) > 0 else 0
df['ObfuscationRatio'] = df['URL'].apply(obfuscation_ratio)

In [18]:
def count_letters(url):
    return sum(c.isalpha() for c in url)

df['NoOfLettersInURL'] = df['URL'].apply(count_letters)


In [19]:
def letter_ratio(url):
    letters = count_letters(url)
    return letters / len(url) if len(url) > 0 else 0

df['LetterRatioInURL'] = df['URL'].apply(letter_ratio)


In [20]:
def count_digits(url):
    return sum(c.isdigit() for c in url)
df['NoOfDigitsInURL'] = df['URL'].apply(count_digits)

In [21]:
def digit_ratio(url):
    digits = count_digits(url)
    return digits / len(url) if len(url) > 0 else 0
df['DegitRatioInURL'] = df['URL'].apply(digit_ratio)

In [22]:
def count_equals(url):
    return url.count("=")
df['NoOfEqualsInURL'] = df['URL'].apply(count_equals)

In [23]:
def count_qmarks(url):
    return url.count("?")
df['NoOfQMarkInURL'] = df['URL'].apply(count_qmarks)

In [24]:
def count_ampersands(url):
    return url.count("&")
df['NoOfAmpersandInURL'] = df['URL'].apply(count_ampersands)

In [25]:
import string

def count_special_chars(url):
    # Define the set of special characters we want to count.
    special_chars = "!@#$%^*()[]{}/\\.:"
    return sum(url.count(c) for c in special_chars)
df['NoOfOtherSpecialCharsInURL'] = df['URL'].apply(count_special_chars)

In [26]:
def special_char_ratio(url):
    special_chars = count_special_chars(url)
    return special_chars / len(url) if len(url) > 0 else 0
df['SpacialCharRatioInURL'] = df['URL'].apply(special_char_ratio)

In [27]:
from urllib.parse import urlparse

def is_https(url):
    # Parse the URL and check if the scheme is "https".
    return 1 if urlparse(url).scheme == "https" else 0
df['IsHTTPS'] = df['URL'].apply(is_https)

In [28]:
X1 = df.loc[:,:'IsHTTPS']

In [29]:
df.tail()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,HasPasswordField,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label,NoOfDigitsInURL
235790,660997.txt,https://www.skincareliving.com,30,www.skincareliving.com,22,0,com,0.133,0.522907,0.100000,...,0,1,51,7,21,187,2,191,1,0
235791,77185.txt,https://www.winchester.gov.uk,29,www.winchester.gov.uk,21,0,uk,0.138,0.028555,0.137931,...,0,0,50,1,7,88,0,31,1,0
235792,622132.txt,https://www.nononsensedesign.be,31,www.nononsensedesign.be,23,0,be,0.129,0.003319,0.161290,...,0,1,27,10,30,58,2,67,1,0
235793,7503962.txt,https://patient-cell-40f5.updatedlogmylogin.wo...,56,patient-cell-40f5.updatedlogmylogin.workers.dev,47,0,dev,0.054,0.000961,0.089286,...,1,0,0,0,3,0,0,0,0,3
235794,384822.txt,https://www.alternativefinland.com,34,www.alternativefinland.com,26,0,com,0.118,0.522907,0.117647,...,0,1,21,6,18,256,0,261,1,0


In [30]:
df.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,HasPasswordField,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label,NoOfDigitsInURL
0,521848.txt,https://www.southbankmosaics.com,32,www.southbankmosaics.com,24,0,com,0.125,0.522907,0.125000,...,0,1,34,20,28,119,0,124,1,0
1,31372.txt,https://www.uni-mainz.de,24,www.uni-mainz.de,16,0,de,0.167,0.032650,0.125000,...,0,1,50,9,8,39,0,217,1,0
2,597387.txt,https://www.voicefmradio.co.uk,30,www.voicefmradio.co.uk,22,0,uk,0.133,0.028555,0.100000,...,0,1,10,2,7,42,2,5,1,0
3,554095.txt,https://www.sfnmjournal.com,27,www.sfnmjournal.com,19,0,com,0.148,0.522907,0.111111,...,0,1,3,27,15,22,1,31,1,0
4,151578.txt,https://www.rewildingargentina.org,34,www.rewildingargentina.org,26,0,org,0.118,0.079963,0.117647,...,0,1,244,15,34,72,1,85,1,0


In [31]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = ["Domain", "TLD", "Title", "URL"]
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Save encoders for later use

In [32]:
df['FILENAME'] = df['FILENAME'].astype(str)
df['FILENAME'] = df['FILENAME'].str.extract(r'(\d+\.?\d*)')
df['FILENAME'] = pd.to_numeric(df['FILENAME'], errors='coerce')
df['FILENAME'].fillna(0, inplace=True)
print(df.dtypes)
print(df)

FILENAME                      float64
URL                             int64
URLLength                       int64
Domain                          int64
DomainLength                    int64
IsDomainIP                      int64
TLD                             int64
CharContinuationRate          float64
TLDLegitimateProb             float64
URLCharProb                   float64
TLDLength                       int64
NoOfSubDomain                   int64
HasObfuscation                  int64
NoOfObfuscatedChar              int64
ObfuscationRatio              float64
NoOfLettersInURL                int64
LetterRatioInURL              float64
NoOfDegitsInURL                 int64
DegitRatioInURL               float64
NoOfEqualsInURL                 int64
NoOfQMarkInURL                  int64
NoOfAmpersandInURL              int64
NoOfOtherSpecialCharsInURL      int64
SpacialCharRatioInURL         float64
IsHTTPS                         int64
LineOfCode                      int64
LargestLineL

C:\Users\vivek\AppData\Local\Temp\ipykernel_17772\2837574320.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['FILENAME'].fillna(0, inplace=True)


In [33]:
Y = df.label

In [34]:
X1 = df.loc[:,:'IsHTTPS']

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X1,Y,test_size= 0.2,random_state=42)

In [36]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(n_estimators=50, random_state=42)
RF.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=50, random_state=42)

In [37]:
RF.score(X_test,Y_test)

0.9998525729579621

In [38]:
X2 =df.loc[:, 'LineOfCode':]
X2 = X2.drop('label',axis = 1)

In [39]:
pip install --upgrade scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import pickle

# Train your model (assuming X_train and Y_train are already defined)
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(n_estimators=50, random_state=42)
RF.fit(X_train, Y_train)

# Save the model to a pickle file
with open('X1.pkl', 'wb') as file:
    pickle.dump(RF, file)

print("Model saved as 'random_forest_model.pkl'")


Model saved as 'random_forest_model.pkl'
